# KNN

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# Parâmetros

QTDE_VIZINHOS = 20
#TAM_AMOSTRA = 100
TAM_AMOSTRA_PERCENTUAL = 0.6  ### sugestão do Fernando em %
QTDE_RODADAS = 10

In [0]:
# Prepara dataframe para comparar resultados de cada rodada

medidasResultado = pd.DataFrame(columns = ['Portfolio',
                                          'Tamanho portfolio',
                                          'Qtde vizinhos',
                                          'Tamanho amostra',
                                          'Número rodada',
                                          'Qtde recomendações total',
                                          'Qtde recomendações únicas',
                                          'Qtde recomendações diferentes total',
                                          'Qtde recomendações diferentes únicas',
                                          'Qtde acertos',
                                          'Qtde acertos únicos',
                                          'Qtde recomendações diferentes e acertos total',
                                          'Qtde recomendações diferentes e acertos únicas'
                                          ])

In [0]:
#df = pd.read_csv("../src/estaticos_market.csv")
#df = df.drop(['Unnamed: 0'], axis=1)

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = "https://drive.google.com/open?id=1YcMHV-9W42-p2NsfiL6BLlmd_ecl8pL7" #shareble link do drive
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Mercado_Formatado_certinho.csv')  

df = pd.read_csv('Mercado_Formatado_certinho.csv', sep=',', decimal='.', encoding = "UTF-8")
df = df.drop(['Unnamed: 0'], axis=1)


1YcMHV-9W42-p2NsfiL6BLlmd_ecl8pL7


### Ajusta modelo

In [0]:
dfSemID = df.drop('id',axis=1)

In [0]:
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=QTDE_VIZINHOS, n_jobs=-1 )

In [0]:
model_knn.fit(dfSemID)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=-1, n_neighbors=20, p=2,
                 radius=1.0)

In [0]:
# Vê saída do modelo para a empresa 0 (índice 0)

model_knn.kneighbors(dfSemID.iloc[[0]])

(array([[1.11022302e-16, 8.92619312e-14, 1.37778677e-13, 1.56430424e-13,
         2.22488694e-13, 2.29705144e-13, 2.36477504e-13, 2.37476705e-13,
         2.60680366e-13, 2.63566946e-13, 2.76667578e-13, 2.81996648e-13,
         2.86770607e-13, 2.86992652e-13, 2.90878432e-13, 2.91655589e-13,
         2.94431146e-13, 2.94653191e-13, 2.99649194e-13, 3.00870440e-13]]),
 array([[     0, 253667, 455035, 355353, 192899, 112746, 343399, 204736,
          40281, 103381,  84091, 272683, 321835, 302060, 336064,  75763,
         443074, 419907, 282108, 187777]]))

In [0]:
# Vê saída do modelo para a empresa 14097 (índice 14097)

model_knn.kneighbors(dfSemID.iloc[[14097]])

(array([[2.22044605e-16, 3.77475828e-15, 4.66293670e-15, 4.99600361e-15,
         5.55111512e-15, 6.10622664e-15, 6.77236045e-15, 7.10542736e-15,
         7.43849426e-15, 8.10462808e-15, 8.10462808e-15, 8.77076189e-15,
         8.88178420e-15, 8.88178420e-15, 8.88178420e-15, 8.99280650e-15,
         9.10382880e-15, 9.43689571e-15, 9.65894031e-15, 9.65894031e-15]]),
 array([[ 14097, 303094,  26491, 227003, 244798, 241432, 135754,  64562,
          94291,  23400, 428239, 159775, 133840, 320729,  79550, 146042,
         372623,  61649, 221448, 320640]]))

In [0]:
# Faz uma lista de 3 listas: empresa, vizinhos de cada empresa, distâncias
# a = [0] * qtdeVizinhos
# b = list(vizinhosdoindividuo[1][0])
# c = list(vizinhosdoindividuo[0][0])
# oi = list(zip(a,b,c))
# pd.DataFrame(data=oi, columns=["Indivíduo","Vizinho","Distância"])

### Funções auxiliareas

In [0]:
# Transforma uma lista de índices em uma lista de IDs

def returnID (df, list_indices):
  list_ids = []

  for i in list_indices:
    list_ids.append(df.iloc[i]['id'])

  return list_ids

In [0]:
# Retorna os vizinhos para cada empresa de uma lista de empresas

def recomenda(lista_empresas, qtde_vizinhos):

  temp = {"Origem":[],"Vizinho":[],"Distância":[]}

  for empresa in lista_empresas:

    vizinhos = model_knn.kneighbors(dfSemID.iloc[[empresa]])

    a = [empresa] * qtde_vizinhos
    b = list(vizinhos[1][0])
    c = list(vizinhos[0][0])
    
    a_ids = returnID(df, a)
    b_ids = returnID(df, b)

    temp['Origem'].extend(a_ids)
    temp['Vizinho'].extend(b_ids)
    temp['Distância'].extend(c)

  # Monta o dataframe e apaga as linhas que é a própria empresa se recomendando
  result = pd.DataFrame(data=temp)
  result = result[result['Origem'] != result['Vizinho']]

  return result

In [0]:
def amostra_portfolio(df_mercado, df_portfolio, tam_amostra):

  # Amostra o portfolio, retornando uma lista de IDs
  amostra_IDs = df_portfolio.sample(n = tam_amostra)['id'].tolist()

  # Pega os índices (na Mercado) dos IDs amostrados
  amostra_indices = df_mercado.index[df_mercado['id'].isin(amostra_IDs)]

  return amostra_indices

In [0]:
# Flag para recomendações fora da amostra
# Adiciona uma coluna no dataframe de recomendações, informando se é Diferente de alguém que já existia na amostra

def flag_recom_diferentes(df_recomendacoes):

  vizinhos = set(list(df_recomendacoes['Vizinho']))
  origens = set(list(df_recomendacoes['Origem']))

  diferentes = list(vizinhos - origens)

  df_recomendacoes['Diferente'] = [1 if i in(diferentes) else 0 for i in list(df_recomendacoes['Vizinho'])]

  return df_recomendacoes

In [0]:
# Flag para acerto
# Adiciona uma coluna no dataframe de recomendações, informando se está no Portfolio

def flag_recom_acerto(df_recomendacoes, df_portfolio):

  vizinhos = set(list(df_recomendacoes['Vizinho']))
  portfolio = set(list(df_portfolio['id']))

  acertos = vizinhos.intersection(portfolio)

  df_recomendacoes['Acerto'] = [1 if i in(acertos) else 0 for i in list(df_recomendacoes['Vizinho'])]

  return df_recomendacoes

In [0]:
# Junta a flag para recomendações fora da amostra e flag para acerto

def flag_recom_diferentes_e_acerto(df_recomendacoes):

  df_recomendacoes['Diferente e Acerto'] = df_recomendacoes['Acerto'] & df_recomendacoes['Diferente']
  
  return df_recomendacoes

In [0]:
# Calcula medidas de cada rodada
#   - Quantidade de recomendações no total
#   - Quantidade de recomendações únicas
#   - Quantidade de recomendações fora da amostra
#   - Quantidade de recomendações fora da amostra únicas
#   - Quantidade de acertos
#   - Quantidade de acertos únicos
#   - Quantidade de recomendações fora da amostra e acertos
#   - Quantidade de recomendações fora da amostra e acertos únicas

def calcula_medidas_rodada(df_recomendacoes):

  qtde_recom_total = len(df_recomendacoes['Vizinho'])
  qtde_recom_unicas = len(df_recomendacoes['Vizinho'].unique())
  qtde_recom_diferentes = df_recomendacoes['Diferente'].sum()
  qtde_recom_diferentes_unicas = len(df_recomendacoes[df_recomendacoes['Diferente'] == 1]['Vizinho'].unique())
  qtde_acertos = df_recomendacoes['Acerto'].sum()
  qtde_acertos_unicos = len(df_recomendacoes[df_recomendacoes['Acerto'] == 1]['Vizinho'].unique())
  qtde_recom_diferentes_e_acertos =  df_recomendacoes['Diferente e Acerto'].sum()
  qtde_recom_diferentes_e_acertos_unicas = len(df_recomendacoes[df_recomendacoes['Diferente e Acerto'] == 1]['Vizinho'].unique())

  return [qtde_recom_total, qtde_recom_unicas, 
          qtde_recom_diferentes, qtde_recom_diferentes_unicas, 
          qtde_acertos, qtde_acertos_unicos,
          qtde_recom_diferentes_e_acertos, qtde_recom_diferentes_e_acertos_unicas]

### Testando portfolio 3

In [0]:
p3 = pd.read_csv('Cópia de estaticos_portfolio3.csv')
p3 = p3.drop(['Unnamed: 0'], axis=1)

In [0]:
TAM_AMOSTRA = round(TAM_AMOSTRA_PERCENTUAL * len(p3))
TAM_AMOSTRA

159

In [0]:
#QTDE_RODADAS = 1

In [0]:
# Para cada rodada

for RODADA in range(QTDE_RODADAS):

  # Amostra o portfolio
  amostra_p3_indices = amostra_portfolio(df, p3, TAM_AMOSTRA)

  # Roda o KNN para aquela amostra
  recomendacoes_p3 = recomenda(amostra_p3_indices, QTDE_VIZINHOS)

  # Flag para recomendações fora da amostra
  recomendacoes_p3 = flag_recom_diferentes(recomendacoes_p3)

  # Flag para acerto
  recomendacoes_p3 = flag_recom_acerto(recomendacoes_p3, p3)

  # Junta a flag para recomendações fora da amostra e flag para acerto
  recomendacoes_p3 = flag_recom_diferentes_e_acerto(recomendacoes_p3)

  # Exporta matriz de recomendações: [Origem, Vizinho, Distância, Diferente, Acerto]
  nomearquivo = 'recomendacoes_p3_rodada_' + str(RODADA) + '.csv'
  recomendacoes_p3.to_csv(nomearquivo)

  # Calcula medidas da rodada
  [qtde_recom_total, qtde_recom_unicas, 
   qtde_recom_diferentes, qtde_recom_diferentes_unicas, 
   qtde_acertos, qtde_acertos_unicos, 
   qtde_recom_diferentes_e_acertos, qtde_recom_diferentes_e_acertos_unicas] = calcula_medidas_rodada(recomendacoes_p3)

  # Adiciona uma linha no dataframe de resultados, com os resultados daquela rodada
  medidasResultado = medidasResultado.append(pd.Series(
                                                        ['Portfolio 3',
                                                        len(p3),
                                                        QTDE_VIZINHOS,
                                                        TAM_AMOSTRA,
                                                        RODADA,
                                                        qtde_recom_total,
                                                        qtde_recom_unicas,
                                                        qtde_recom_diferentes,
                                                        qtde_recom_diferentes_unicas,
                                                        qtde_acertos,
                                                        qtde_acertos_unicos,
                                                        qtde_recom_diferentes_e_acertos,
                                                        qtde_recom_diferentes_e_acertos_unicas
                                                        ],
                                                        index = medidasResultado.columns
                                                        ), ignore_index=True)

In [0]:
# Exporta matriz de resultados de todas as rodadas
nomearquivo = 'resumo_rodadas_p3.csv'
medidasResultado.to_csv(nomearquivo)

In [0]:
#medidasResultado.head(10)

In [0]:
recomendacoes_p3.head()

,Origem,Vizinho,Distância,Diferente,Acerto,Diferente e Acerto
1,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...,e5ac842358b2fd0a6449ed3abe025b1c5a5470e2b4cd24...,3.494211e-09,1,0,0
2,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...,d27c815e7880827e4b96fb5c8729a0fc403c55b2f64b3d...,5.092978e-09,1,0,0
3,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...,faa7f1a200e3b2e6da7334ffaef18ff3faf618fa980d16...,6.720639e-09,0,1,0
4,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...,25a22502f77a075af92878f748a598fee97116c1f16f7e...,9.542350e-09,1,0,0
5,bb61c09449a1c1f8b78955d8b7a7f73599e78358993fe2...,3eb09adfd5c81e02b960584ca51cc699998d748aaf1fee...,1.943159e-08,1,0,0


### Testando portfolio 2

In [0]:
p2 = pd.read_csv('Cópia de estaticos_portfolio2.csv')
p2 = p2.drop(['Unnamed: 0'], axis=1)

In [0]:
TAM_AMOSTRA = round(TAM_AMOSTRA_PERCENTUAL * len(p2))
TAM_AMOSTRA

340

In [0]:
# Para cada rodada

for RODADA in range(QTDE_RODADAS):

  # Amostra o portfolio
  amostra_p2_indices = amostra_portfolio(df, p2, TAM_AMOSTRA)

  # Roda o KNN para aquela amostra
  recomendacoes_p2 = recomenda(amostra_p2_indices, QTDE_VIZINHOS)

  # Flag para recomendações fora da amostra
  recomendacoes_p2 = flag_recom_diferentes(recomendacoes_p2)

  # Flag para acerto
  recomendacoes_p2 = flag_recom_acerto(recomendacoes_p2, p2)

  # Junta a flag para recomendações fora da amostra e flag para acerto
  recomendacoes_p2 = flag_recom_diferentes_e_acerto(recomendacoes_p2)

  # Exporta matriz de recomendações: [Origem, Vizinho, Distância, Diferente, Acerto]
  nomearquivo = 'recomendacoes_p2_rodada_' + str(RODADA) + '.csv'
  recomendacoes_p2.to_csv(nomearquivo)

  # Calcula medidas da rodada
  [qtde_recom_total, qtde_recom_unicas, 
   qtde_recom_diferentes, qtde_recom_diferentes_unicas, 
   qtde_acertos, qtde_acertos_unicos, 
   qtde_recom_diferentes_e_acertos, qtde_recom_diferentes_e_acertos_unicas] = calcula_medidas_rodada(recomendacoes_p2)

  # Adiciona uma linha no dataframe de resultados, com os resultados daquela rodada
  medidasResultado = medidasResultado.append(pd.Series(
                                                        ['Portfolio 2',
                                                        len(p2),
                                                        QTDE_VIZINHOS,
                                                        TAM_AMOSTRA,
                                                        RODADA,
                                                        qtde_recom_total,
                                                        qtde_recom_unicas,
                                                        qtde_recom_diferentes,
                                                        qtde_recom_diferentes_unicas,
                                                        qtde_acertos,
                                                        qtde_acertos_unicos,
                                                        qtde_recom_diferentes_e_acertos,
                                                        qtde_recom_diferentes_e_acertos_unicas
                                                        ],
                                                        index = medidasResultado.columns
                                                        ), ignore_index=True)

In [0]:
# Exporta matriz de resultados de todas as rodadas
nomearquivo = 'resumo_rodadas_p2.csv'
medidasResultado.to_csv(nomearquivo)

In [0]:
medidasResultado.tail(10)

,Portfolio,Tamanho portfolio,Qtde vizinhos,Tamanho amostra,Número rodada,Qtde recomendações total,Qtde recomendações únicas,Qtde recomendações diferentes total,Qtde recomendações diferentes únicas,Qtde acertos,Qtde acertos únicos,Qtde recomendações diferentes e acertos total,Qtde recomendações diferentes e acertos únicas
10,Portfolio 2,566,20,340,0,6460,2610,5391,2339,1707,440,638,169
11,Portfolio 2,566,20,340,1,6460,2665,5370,2406,1729,426,639,167
12,Portfolio 2,566,20,340,2,6460,2834,5535,2574,1624,439,699,179
13,Portfolio 2,566,20,340,3,6460,2709,5399,2449,1730,436,669,176
14,Portfolio 2,566,20,340,4,6460,2806,5517,2542,1624,444,681,180
15,Portfolio 2,566,20,340,5,6460,2795,5457,2540,1681,433,678,178
16,Portfolio 2,566,20,340,6,6460,2719,5413,2460,1731,432,684,173
17,Portfolio 2,566,20,340,7,6460,2812,5439,2557,1704,433,683,178
18,Portfolio 2,566,20,340,8,6460,2902,5541,2647,1614,433,695,178
19,Portfolio 2,566,20,340,9,6460,2776,5433,2517,1712,435,685,176


### Testando portfolio 1

In [0]:
p1 = pd.read_csv('Cópia de estaticos_portfolio2.csv')
p1 = p1.drop(['Unnamed: 0'], axis=1)

In [0]:
p1 = p1.filter(['id'])

In [0]:
TAM_AMOSTRA = round(TAM_AMOSTRA_PERCENTUAL * len(p1))
TAM_AMOSTRA

340

In [0]:
# Para cada rodada

for RODADA in range(QTDE_RODADAS):

  # Amostra o portfolio
  amostra_p1_indices = amostra_portfolio(df, p1, TAM_AMOSTRA)

  # Roda o KNN para aquela amostra
  recomendacoes_p1 = recomenda(amostra_p1_indices, QTDE_VIZINHOS)

  # Flag para recomendações fora da amostra
  recomendacoes_p1 = flag_recom_diferentes(recomendacoes_p1)

  # Flag para acerto
  recomendacoes_p1 = flag_recom_acerto(recomendacoes_p1, p1)

  # Junta a flag para recomendações fora da amostra e flag para acerto
  recomendacoes_p1 = flag_recom_diferentes_e_acerto(recomendacoes_p1)

  # Exporta matriz de recomendações: [Origem, Vizinho, Distância, Diferente, Acerto]
  nomearquivo = 'recomendacoes_p1_rodada_' + str(RODADA) + '.csv'
  recomendacoes_p1.to_csv(nomearquivo)

  # Calcula medidas da rodada
  [qtde_recom_total, qtde_recom_unicas, 
   qtde_recom_diferentes, qtde_recom_diferentes_unicas, 
   qtde_acertos, qtde_acertos_unicos, 
   qtde_recom_diferentes_e_acertos, qtde_recom_diferentes_e_acertos_unicas] = calcula_medidas_rodada(recomendacoes_p1)

  # Adiciona uma linha no dataframe de resultados, com os resultados daquela rodada
  medidasResultado = medidasResultado.append(pd.Series(
                                                        ['Portfolio 1',
                                                        len(p1),
                                                        QTDE_VIZINHOS,
                                                        TAM_AMOSTRA,
                                                        RODADA,
                                                        qtde_recom_total,
                                                        qtde_recom_unicas,
                                                        qtde_recom_diferentes,
                                                        qtde_recom_diferentes_unicas,
                                                        qtde_acertos,
                                                        qtde_acertos_unicos,
                                                        qtde_recom_diferentes_e_acertos,
                                                        qtde_recom_diferentes_e_acertos_unicas
                                                        ],
                                                        index = medidasResultado.columns
                                                        ), ignore_index=True)

In [0]:
# Exporta matriz de resultados de todas as rodadas
nomearquivo = 'resumo_rodadas_p1.csv'
medidasResultado.to_csv(nomearquivo)

In [0]:
medidasResultado.tail(10)